<a href="https://colab.research.google.com/github/samina-if/AdventOfCode2024/blob/main/Advent_0f_Code_Day16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Part 1
# Import necessary libraries
from collections import deque
from typing import List, Set, Tuple

class MazeSolver:
    def __init__(self, maze: List[List[str]]):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
        self.start, self.end = self._find_start_end()

    def _find_start_end(self) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        start = end = None
        for y in range(self.height):
            for x in range(self.width):
                if self.maze[y][x] == 'S':
                    start = (y, x)
                elif self.maze[y][x] == 'E':
                    end = (y, x)
        return start, end

    def find_lowest_score(self) -> int:
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up
        best_scores = {}  # (y, x, dir) -> min_score
        queue = deque([(self.start[0], self.start[1], 0, 0)])  # y, x, dir, score
        min_end_score = float('inf')

        while queue:
            y, x, dir, score = queue.popleft()

            if score >= min_end_score:
                continue

            state = (y, x, dir)
            if state in best_scores and best_scores[state] <= score:
                continue
            best_scores[state] = score

            if (y, x) == self.end:
                min_end_score = min(min_end_score, score)
                continue

            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and self.maze[ny][nx] != '#'):
                queue.append((ny, nx, dir, score + 1))

            queue.append((y, x, (dir - 1) % 4, score + 1000))
            queue.append((y, x, (dir + 1) % 4, score + 1000))

        return min_end_score

    def get_optimal_path(self) -> Set[Tuple[int, int]]:
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up
        best_scores = {}  # (y, x, dir) -> min_score
        queue = deque([(self.start[0], self.start[1], 0, 0, {(self.start[0], self.start[1])})])
        optimal_path = set()

        while queue:
            y, x, dir, score, path = queue.popleft()

            if score > best_scores.get((y, x, dir), float('inf')):
                continue

            if (y, x) == self.end:
                optimal_path = path
                break

            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and self.maze[ny][nx] != '#'):
                new_path = path | {(ny, nx)}
                queue.append((ny, nx, dir, score + 1, new_path))

            queue.append((y, x, (dir - 1) % 4, score + 1000, path.copy()))
            queue.append((y, x, (dir + 1) % 4, score + 1000, path.copy()))

        return optimal_path

def visualize_path(maze: List[List[str]], optimal_tiles: Set[Tuple[int, int]]) -> None:
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if (y, x) in optimal_tiles:
                if maze[y][x] in ['S', 'E']:
                    print(maze[y][x], end='')
                else:
                    print('O', end='')
            else:
                print(maze[y][x], end='')
        print()

def main():
    # Read maze input from input.txt
    with open('input.txt', 'r') as file:
        maze = [list(line.strip()) for line in file.readlines()]

    solver = MazeSolver(maze)
    lowest_score = solver.find_lowest_score()
    print(f"Lowest score to reach the end: {lowest_score}")

    # Visualize the optimal path
    optimal_tiles = solver.get_optimal_path()
    print("\nOptimal Path Visualization:")
    visualize_path(maze, optimal_tiles)

if __name__ == "__main__":
    main()


Lowest score to reach the end: 73404


In [1]:
#Part 2
from collections import deque
from typing import List, Set, Tuple

class MazeSolver:
    def __init__(self, maze: List[List[str]]):
        self.maze = maze
        self.height = len(maze)
        self.width = len(maze[0])
        self.start, self.end = self._find_start_end()

    def _find_start_end(self) -> Tuple[Tuple[int, int], Tuple[int, int]]:
        start = end = None
        for y in range(self.height):
            for x in range(self.width):
                if self.maze[y][x] == 'S':
                    start = (y, x)
                elif self.maze[y][x] == 'E':
                    end = (y, x)
        return start, end

    def find_optimal_tiles(self) -> int:
        # Directions: right, down, left, up
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

        # First pass: Find minimum score to end from each position and direction
        best_scores = {}  # (y, x, dir) -> min_score
        queue = deque([(self.start[0], self.start[1], 0, 0)])  # y, x, dir, score
        min_end_score = float('inf')

        while queue:
            y, x, dir, score = queue.popleft()

            if score >= min_end_score:
                continue

            state = (y, x, dir)
            if state in best_scores and best_scores[state] <= score:
                continue
            best_scores[state] = score

            # Found end
            if (y, x) == self.end:
                min_end_score = min(min_end_score, score)
                continue

            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and
                self.maze[ny][nx] != '#'):
                queue.append((ny, nx, dir, score + 1))

            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000))
            queue.append((y, x, (dir + 1) % 4, score + 1000))

        # Second pass: Find tiles that are part of optimal paths
        optimal_tiles = set()
        visited = set()
        queue = deque([(self.start[0], self.start[1], 0, 0, {(self.start[0], self.start[1])})])

        while queue:
            y, x, dir, score, path = queue.popleft()

            if score > min_end_score:
                continue

            state = (y, x, dir)
            if score > best_scores.get(state, float('inf')):
                continue

            if (y, x) == self.end and score == min_end_score:
                optimal_tiles.update(path)
                continue

            # Try moving forward
            ny, nx = y + directions[dir][0], x + directions[dir][1]
            if (0 <= ny < self.height and 0 <= nx < self.width and
                self.maze[ny][nx] != '#'):
                new_path = path | {(ny, nx)}
                queue.append((ny, nx, dir, score + 1, new_path))

            # Try turning
            queue.append((y, x, (dir - 1) % 4, score + 1000, path.copy()))
            queue.append((y, x, (dir + 1) % 4, score + 1000, path.copy()))

        return len(optimal_tiles)

def visualize_path(maze: List[List[str]], optimal_tiles: Set[Tuple[int, int]]) -> None:
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if (y, x) in optimal_tiles:
                if maze[y][x] in ['S', 'E']:
                    print(maze[y][x], end='')
                else:
                    print('O', end='')
            else:
                print(maze[y][x], end='')
        print()

def main():
    sample_maze = """#############################################################################################################################################
#.#.........#...#.....................#...........#...............#...#.................#...........#...#...#.....#.#...........#.....#....E#
#.#.#.#######.#.#.#########.#########.###.#.#####.###.#####.#.###.#.#.#########.#####.#.#####.#.#####.#.#.#.#.###.#.#.#####.#.#.#####.#.###.#
#...#.........#.....#.........#.....#.......#...#.........#.#...#...#...#...#...#.#.....#...#.#.......#...#.#...#.#...#.....#.#.#.....#.#...#
#.###################.###.###.#.###.#####.#####.#####.###.###.#.#######.#.#.#.###.#.#.###.#.###.###########.###.#.#####.###.#.#.#.#.###.#.#.#
#.#.....#.....#...#...#.......#.#.#.#...........#.......#...#...#.....#...#...#.#...#.....#...#.........#...#.#.#...#...#.....#.#.#.#...#...#
#.#.###.#.#####.#.#.#####.###.#.#.#.#.#.#.#####.#.###.#.###.#.#######.#######.#.#.#.#.#.#####.#########.#.#.#.#.###.#.###.###.#.#.###.#####.#
#.#.#.....#.....#.#.#...#.#...#.#.#...#.#.....#.#.#.....#...#.#.....#...........#.#...#.#.............#.#.#...#.#.#...#.....#.#.#...#.#...#.#
#.#.#######.#####.#.#.#.###.#.#.#.#####.###.#.#.###.#.###.###.#.###.#.###.###.#.#.#.#.#.#########.###.###.#.###.#.#######.#.###.#.#.#.#.#.#.#
#.#...#...#...#.....#.#.....#.#.#.....#...#.#.#.#...#.#...#.....#...#.#.#...............#...........#.....#.#...#.#.....#.#.#...#.#.....#.#.#
#.###.#.#.###.#.#####.#######.#.###.#.###.#.#.#.#.#.###.###.#.###.###.#.#####.#.###.#.###.#####.#.#.#######.#.###.#.#.#.###.#.###########.#.#
#.....#.#.....#.#.....#.....#.#.#...#...#.#.#.....#.#...#.....#...#.#.........................................#.....#.#...#.#...#...#.....#.#
#####.#.#####.###.#######.#.#.#.#.#######.#########.#.#######.#.###.#####.#.#.#.#.#.#.#######.#.#.#.#.#.#####.#.#####.###.#.###.#.#.#.#####.#
#.....#.....#.#...#.......#.#.#.#...#.....#...#...#.#.#.....#.#.#.#.....#...#.....#.#.#.....#.#.#.#.#.#.....#.#.#...#.#.#.#...#...#.#.#.....#
#.#########.###.#####.#######.#.#.#.#.#####.#.#.#.###.#.###.#.#.#.#.###.###.#.#.#.#.#.#.###.###.#.#.#.#####.#.#.#.#.#.#.#.#.#######.###.#.#.#
#.#...#.....#...#.....#.....#.#.#.#.#.#...#.#...#.......#.#.....#...#.#.#...#.#.#.....#.#.#.....#...#.#.....#.#...#.#.#...#.......#.....#...#
#.#.#.#.#####.###.#####.###.#.#.###.#.#.#.#.#######.#####.#######.###.#.#.###.#.###.###.#.#######.#.#.###.###.###.###.#.#####.#############.#
#.#.#...#.....#...#.....#.#...#...#.#...#.#...#.....#.....#.......#...#.#.#...#...#...#.#...#.......#...#.#...#.#.#...#.......#.....#...#...#
###.###.#.#######.#.#####.#######.#.###.#.#.###.###.#.###.#.#########.#.#.#.###.###.#.#.#.###.#########.#.###.#.#.#.###########.#.#.###.#.###
#...#...#.#...#...#...#...#.....#.......#...#...#...#...#.#.........#.#...#.....#...#.#.#.#...#.........#.....#...#.....#.......#.#...#.#...#
#.###.#.#.#.#.#.#.###.#.###.###.#######.#####.###.###.###.#########.#.#####.#####.###.#.#.#.###.###############.#######.#.#######.###.#.###.#
#.....#.....#...#...#.#...#.#.....#.....#.....#...#...#.......#...#.........#...#.#.#...#.#.#.#.#.............#.#.....#...#.....#.#.#...#...#
#.#############.#####.#.#.#.#.#####.#####.#####.#######.#####.###.#######.#.#.#.#.#.#####.#.#.#.#.#.#.#######.#.#.#.###########.#.#.#.###.###
#...............#.....#.#...#.....#.....#.#...#...#.....#...#.....#.#.....#.#.#...#.....#...#.#.#.#.#.#.#.....#.#.#...........#.#.#...#.....#
#####.#.#.#.#.###.#####.#########.###.###.#.#.###.#.#####.#.#####.#.#.#####.#.#####.#.#.#.###.#.#.#.#.#.#.###.#.#########.###.#.#.###.#.###.#
#.......#.#.#...#.#.......#...#.....#.#...#.#...#...#.....#.....#...#...#...#.....#.#.#.#.#...#...#.#...#.#...#.#.........#.#...#...#.#.#...#
#######.###.###.#.#.#.###.#.#.#.#####.#.###.###.#########.#.#.#####.###.###.#####.#.#.###.#.#.###.#.#.###.#.###.#.#########.###.###.###.#.#.#
#...........#...#...#.#.....#.#.......#.......#...#.....#.#.#.#.....#.#...#...#...#.#.#...#.#...#.#.#.#.......#.#.#.......................#.#
#.#####.#####.#.#.###.#######.#############.#####.#.###.#.#.#.#.#####.###.#####.###.#.#.###.#.###.#.#.#.###.###.#.#.#########.#.#.###.#####.#
#.#.........#.....#.#...#...#.............#.#.....#...#...#.#.#.#.......#...#...#.#.#.#.#.#.......#.#.#.#...#...#.#.#.......#.....#.........#
#.#####.#.###.#.###.###.###.###########.#.#.#.#.#.###.#.###.#.#.#.#.#######.#.###.#.#.#.#.#.#.#.###.#.#.#.###.###.###.#####.#####.#.#.#####.#
#.#.......#...#.......#.#.................#.#.#.#.#...#.#...#.#.#.#.#.....#...#.#...#...#.#.#.#.#.....#.....#.#.#...#...#.#.....#...#.....#.#
#.#.###.###.###########.#.#######.#####.###.#.#.#.#.###.#.###.#.###.#.###.#####.#.#######.#.#.#.#.#.###.###.#.#.#.#.###.#.#####.#.#######.#.#
#.#.........#.........#.#...#...#.#.....#...#.#...#.......#...#.....#.#...#.....#.#.....#...#.#.#.....#...#.#.#.#.....#.#.....#...#.......#.#
#.#.###.#####.#.#####.#.#.#.#.###.#.#####.###.#######.#####.#####.###.#.###.###.#.#.###.#.#.#.#.#######.#.###.#.#.###.#.#.###.###.#.#######.#
#.#.......#...#...#...#...#.#.....#...#.....#.#.......#.....#.#...#...#...#.#...#.#...#...#.#.#.......#.#.....#...#...#.#.#.......#.....#...#
#.#######.#######.#.#####.#.#########.#.###.#.#.###.###.#####.#.###.#####.#.#.#.#.#######.#.#.#######.#.#######.###.###.###.#####.#####.#####
#.......#.........#.#.....#...........#...#.#.#.#.#.#.........#.#...#.....#.#.#.#.#.....#...#.......#.#.....#.....#...#.....#.........#.....#
#.#####.###.#######.#.###.#.#.#######.###.#.#.#.#.#.###########.#.###.#####.#.###.#.###.###.#.###.###.#####.#.###.###.#####.#.###.###.#.###.#
#.#.#...#...#...#.#...#...........#.......#.#.#...#.#.....#.....#...#...#.........#...#.#...#...#.#...#.#...#.#.....#.....#.#.#.....#.#.#...#
#.#.#.###.###.#.#.#####.###.#####.#.#######.#.###.#.#.###.#.#######.###.#.###.#####.#.#.#.#####.###.###.#.###.#.#######.###.#.#.#.###.#.#.#.#
#...#.#...#...#.#.......#.....#...#...#...#...#...#.#...#...#...#...#.#...#.........#...#.........#.#.#...#.....#.....#.....#...............#
###.#.#####.###.#.###.#########.###.###.#.#.#.#####.###.#####.#.#.###.#####.#########.###########.#.#.#.###.#####.###.#.#####.#####.#.###.#.#
#...............#...#.#.........#...#...#.....#.....#...#.....#.#...#.....#...#...#...#.........#.#.#.....#...#...#...#.....#.....#.......#.#
#.###.#.#######.#.#.###.#########.###.#####.#.#.#####.#####.#.#####.#.#.###.#.#.#.#.###.#.#####.#.#.#####.#.###.###.#####.#.#.#.#.#########.#
#...#.#.#...#...#.#.....#...........#.#...#.#.#.#...#.#.....#.#.....#.#.#...#...#.#.#...#.#...#...#...#...#.#...#.#.......#.#.#.#.....#.....#
#.###.#.###.#.#.#####.###############.#.#.###.#.#.#.#.###.#.###.#####.#.#.#######.#.#####.#.#.#######.#.###.#.###.###.#.###.#.#.#.###.#######
#.#...#.#...#.#.#...#.......#.........#.#.....#...#.#.#...#.#.......#.#...#.....#.#...#...#.#.#...#...#.....#...#.....#.....#...#...#...#...#
#.#.#.#.#.###.###.#.#######.#.#######.#.#######.###.#.#.#####.###.#.#.#####.###.#.###.#.#.#.#.#.#.#.#.#.#######.#.#####.#######.#.#.###.#.#.#
#.#.#.#.#...#.....#.....#.....#.......#.#.....#.#.#.....#.....#...#...#.....#...#.#.#...#.#.#.....#.#...#.......#.......#.........#...#...#.#
#.#.###.###.#####.#####.#######.#####.#.#.#####.#.#.###.#.#####.#.#####.#.###.###.#.#####.#.#####.#.#.###.###.###.#######.#.###.#####.#####.#
#.#...#.....#.........#.......#.#...#.#.#.......#.......#.....#.........#...#.#...#.#...#.#...#...#.#.....#...#...#...#...#.#.....#.#.....#.#
#.###.#####.#.###.#####.#####.###.#.###.#.#.#####.#.###.#####.#############.#.#.###.#.#.#.###.#.#.#.#########.#.###.#.#.#.###.###.#.###.###.#
#...#.......................#.#...#...#.#...#...#.#.#.#.#.#...#.......#.....#.#.#...#.#.....#.#.#.....#.....#.#.#...#...#.....#.#.#.....#...#
###.###.#.###.#.###.#.#.###.#.#.###.#.#.###.#.#.###.#.#.#.#.###.#####.#.#####.#.###.#.#######.#.#####.#.###.#.#.#.#############.#.#######.#.#
#.#...#...#.......#...#.#...#...#.#...#...#...#.....#.#...#.....#.#...#.....#.#...#.#.....#...#.......#...#...#.#...#...........#.#.....#.#.#
#.###.#####.#####.#####.#########.#.#####.###########.###.#######.#.#####.#.#.###.#.###.###.###########.#.#####.###.#.#.#######.#.#.###.#.#.#
#...#.#.......#...#...#...#.......#.....#...#.#.........#.......#.#.#...#.#...#...#.#...#...#.........#.#.....#.#...#.#...#.........#.#...#.#
###.#.#.#####.#.###.#.#.#.#.#.#########.###.#.#.#####.#########.#.#.#.#.#######.###.#.###.#######.###.#######.#.#.#.#.#.#.#.###.#.###.#####.#
#...#.#.#...#.#...#.#.#.#...#.#.....#...#...#.......#.#.........#.#...#.......#.#...#...#...#.....#.........#.#...#.#.#.#.#.#.....#.........#
#.###.#.#.#.#.#####.#.#.#####.#.###.#.###.#####.#####.#.#########.#####.#####.#.#.#.###.###.#.#####.#######.#.#####.#.###.#.###.#.#.#######.#
#.#...#.#.#...#.....#.#.#.....#.#.#...#...#.....#.....#.#.........#.........#...#.#.....#.#.#.....#.#...............#.#.......#.#...#.......#
#.#.###.#.#####.#####.###.#.###.#.#####.#.#.###.#.#####.###.#####.#########.#####.#.#.###.#.#####.#.#.#.###########.#.#.#.###.#.#.###.###.#.#
#...#...#.....#.#.......#.#.#...#.....#.#...#...#.....#.....#...#...........#.....#.#.....#.#...#.#.#.#.#.....#.....#...#...#.#.#.#...#.#.#.#
#.###.#######.#.#######.#.#.#.#####.###.#####.###.###.#####.#.#.###.#######.#.###.#######.#.#.#.#.###.#.#.###.#.#####.#####.#.#.#.#.###.#.#.#
#.....#.#.....#.#.........#.#...#...#...#...#...#...#.....#.#.#...#.#...#...#...#.......#.#...#.#.....#.#.#...#.#.......#...#.#.#.#.#...#...#
#######.#.#####.#.###.#####.###.#.#.#.#####.#.#.#####.#.###.#.###.###.#.#####.#####.#.#.#.#####.#####.#.#.#.#####.#.###.#.#.#.#.#.#.#.#####.#
#.......#.#.....#...#...#...#...#.#.#.#.....#.#.....#.#.....#.#.#.....#.......#...#...#...#.....#...#.#.#.#.#.....#.#...#.#.#.#.#.#...#.....#
#.#####.#.#.#######.###.#####.###.###.#.###.#.#####.#########.#.#.#.#.#########.#.###.#####.#####.#.###.#.#.#.#####.#.###.#.#.#.#.#####.###.#
#...#...#.#.......#...#...#...#.......#.#.#...#...#...........#...#.............#...#.#.....#.....#.....#.#.......#.#.#.....#.#.#.......#...#
###.#.###.#.#.#.#.###.###.#.#########.#.#.###.###.#############.#.#########.#.#####.#.#.#####.#######.###.#########.###.#####.#.#.#######.#.#
#.#.#.....#...#.#...#...#...#...........#...#.....#...#.....#...#...#...#...#...#...#...#.......#...#.#...#.........#.....#...#.....#...#.#.#
#.#.#########.#.#######.#####.#####.#.#.#.#####.#.#.#.#.#.#.#.#####.#.#####.###.#.###.#####.###.#.###.#.###.#.#####.#.#####.#####.#.#.###.#.#
#...#.........#.......#.....#.....#.........#...#...#...#.#.#.....#.#.#...#.#.#.#.....#...#...#.#.....#...#.#.#...#.#.#.....#...#.....#...#.#
#.#######.###.#####.#.#.###.#####.#.#####.#.#.###.#######.#.###.#.#.#.#.#.#.#.#.#######.#.#####.#.#######.#.#.#.#.###.#.#####.#.#####.#.###.#
#.......#.....#...#.#.....#.#...#.#.....#.#...#...#...#.......#.#...#...#.#...#.#.......#.......#.......#.#.....#.#...#.......#...#...#.....#
#######.#.###.#.#.###.#####.#.#.#.#####.#.#####.#####.#.#####.#.#.#######.###.#.#.#################.#####.#####.#.#.###########.#.#########.#
#.#.............#.....#...#.#.#.......#.#...#.........#.....#.#.#.......#...#.#.#...#...........#...#.....#...#.#.#.#.......#...#.........#.#
#.#.#######.#.#.#####.#.#.#.#.###.#####.###.###########.###.#.#.#####.#.###.###.#.#.###.#.###.#.#.###.#####.#.###.#.###.###.###.#########.#.#
#.#...#...#.#...#.....#.#...#.#...#.....#.#...#...#...#.#...#.#.#.#...#.#...#...#.#...#.#.#.....#.#...#.#...#.....#...#.#...........#...#...#
#.###.#.#.#.#.#.#######.#####.#.###.#####.###.#.#.#.#.#.#.#.###.#.#.###.#.###.###.###.###.#######.#.###.#.#.#########.#.#######.###.#.#.#####
#...#...#.#.#...#.....#.......#...#.#...........#...#.#.#.#.#...#.#.#...#.....#.....#...#.....#...#.....#.#.#...#...#.#.........#.#.#.#.....#
#.#.#####.#.#.#.#.###.###########.#.#.###############.#.#.###.###.#.#################.#.#.###.#.#######.#.###.#.#.#.#.###.#####.#.#.#.#.###.#
#.#.#.....#...#.#.#.#.............#.#.....#.....#...#.#.#...#.....#.#.................#.#.#.#...#.....#.#.....#...#.#...#...#.#.#...#.#.#...#
#.#.#.#####.#.###.#.#####.#######.#.#######.###.#.#.#.#####.#.###.#.#.#######.#######.#.#.#.#####.#.#.#.###########.###.###.#.#.#.###.#.###.#
#.#.#.#.....#...........#.#.#.....#.#.........#.#.#.#.#...#...#...#.#.#.......#.....#.#.......#...#.#.#.#.........#...#...#...#.#...#.#...#.#
#.###.#.#####.#####.#####.#.#.#####.#.#########.#.#.#.#.#.#.#.#.###.#.#.#.#####.###.#.#.#######.###.###.#.#####.###.#####.###.#.###.#.###.###
#.#...#.#.........#.#.....#...#...#.....#...#...#...#...#.#.#...#.#.#...#.#.............#.........#.#...#...#.#...#.....#.#...#...#.#...#...#
#.#.#####.#######.#.#.#####.###.#########.#.#.###.#.#####.#####.#.#.###.#.###.#########.#.#########.#.#####.#.###.#####.#.#.###.#.#.#######.#
#...............#.#.#...#.#.#.......#.....#...#...#.....#.....#...#.#...#...#...#.....#.#.......#.......#.#.....#.....#.#.#...#.#.#.#.......#
#.#####.###.#.#.#.###.#.#.#.#######.#.#.#######.#.#.#########.#####.#.#####.#####.###.#########.#.#####.#.#####.###.#.#.#.###.#.#.#.#.#####.#
#.....#...#.#...#...#.#.#.#.........#.#.#.....#...#.#.............#.#.#...#.......#.#.......#...#.....#.......#...#...#...#.#...#.#.#.#...#.#
###.#.###.#.###.###.#.#.#.#######.###.#.#.###.#.#.#.#.#####.#####.#.#.#.#####.###.#.#####.###.###.###.#.#.#.#####.#.#######.#.#.#.#.#.#.#.#.#
#...#.#.#...#...#...#.#.#...#.....#...#...#.#...#.#...#.#...#.....#.#.#...........#.#...#...#...#...#.#.#.#.......#.......#...#...#.#.#.#.#.#
#.###.#.#####.#.#.#####.#.###.#####.###.###.#####.#.###.#.###.#####.#.#.#####.###.#.#.#.###.###.#.###.#.#.###.###.###.###.#.###.###.#.#.#.#.#
#...#.#.......#.#...#...#.........#...#...........#.....#...........#.#.#...#.#...#.#.....#.........#...#...#.......#.#...#.#.#.#...#.#.#...#
###.#.#.###.###.#.#.#.###.#######.###.#####.#####.#.#########.#######.#.#.#.#.###.#.#.#.#.#########.#######.#######.#.#.#.#.#.#.#.###.#.#####
#.............#...#.#...#.#...#.#.#.#.......#.....#.#...........#...#.#.#.#.#...#.#...#.#.#.......#...#.......#...#.#...#.....#.#.#...#.#...#
#.###.#####.#.###.#.#.#.#.#.#.#.#.#.#########.#####.#.#########.#.#.#.#.#.#.###.#.#####.###.#.###.###.#.#.#####.#.###.#####.###.#.#.#####.#.#
#.#.#...#...#...#.#.#.#.#.#.#.#...#...#.......#.....#...#.....#.#.#...#.#.#.#...#.....#.....#.#.......#.....#...#...#.#...#.#...#...#.....#.#
#.#.###.#.###.#.#.#.#.#.###.#.#.#####.#.#.#############.#.#####.#.#####.#.#.#.#######.#.#####.#############.#.#####.###.#.#.#.#######.#####.#
#.#.#.....#...#.....#.#.....#.#.#.....#.#.......#.....#.#...#...#.#...#.#.#.#.#...#.#.#.....#...............#.....#.....#.#.#...#.....#...#.#
#.#.#.#.###########.#.#######.#.###.###.###.###.#.#.###.###.#.###.#.#.#.###.#.#.#.#.#.#####.#####################.#######.#.###.#.#####.#.#.#
#.#.....#...........#...#.....#.#...#...#.....#...#...#.....#...#...#.#...#.#.#.#.#.#...#.#...#.......#.........#...#...#.#...#.....#...#...#
#.#####.#.###############.#####.#.#.#.#.#.###########.###.#####.#####.###.#.#.#.#.#.###.#.#.###.#####.#.#######.###.#.#.#.#########.#.#.#.#.#
#.....#.#.#.........#.....#...#.#.#.........#.......#.#...#.......#...#...#.#.#.#.....#.#...#...#...#...#.....#.....#.#.#.....#...#.#.#...#.#
#.###.#.#.#.#######.#.#####.#.#.#.###.###.###.#####.#.#.###.#######.#####.#.#.#.###.###.#.###.#####.#####.###.#########.#####.#.#.#.#.#.#.#.#
#...#.#.#...#...#.#...#.....#.#.#.#.....#...#.#...#...#.#...#.......#...#.#.#.#...#.#...#.#...#.....#...#.#...#.........#...#...#.#...#.....#
#####.#.#####.#.#.#####.###.#.#.#.#.#.#.#.#.#.#.#.###.#.#.#.#.#######.#.#.#.#.###.#.#.#####.###.#.#.#.#.#.#.###.###.#.#.#.#.#####.#.###.#.#.#
#.....#...#...#.#.....#.#...#.#.#...#.#.#.#...#.#.....#.#.#.....#.....#.#...#.#...#.#.......#...#.#.#.#.#.#...#...#.#.....#...#...#.#.....#.#
#.#####.#.###.#.#.#.###.#.#####.#####.#.#.#####.#######.#.#####.#.#####.#.###.#.#####.#.#.#.#####.###.#.#.###.###.#.###.#######.#####.#.#.#.#
#.#...#.......#...#.#...#.......#.....#.#...#.#.#.......#.....#.#...#.#.#.#...#.....#...#.#.....#.....#.......#...#...#.#.......#.....#.#.#.#
#.###.#.#############.###########.#####.###.#.#.#.#########.#.#.###.#.#.###.#.#.###.#####.#####.#.###########.#.#.###.###.#######.#####.#.#.#
#...#...#.....#.....#...#.........#.#...#...#...#.#.......#...#.....#.#.....#.....#.....#...#...#.#...........#.#...#.#...#.......#.....#...#
###.###.#.###.#.###.###.#####.#####.#.###.###.###.###.#.#.#.#####.#.#.#######.#######.#.###.#.#####.#########.#.###.#.#.###.#.#######.#####.#
#...#.......#...#...#...#...#.#.......#...#.....#...#.#.#.#.....#.#.#.#.......#.#.....#...#...#.....#...#.....#...#.#...#...................#
#.#.#.#.#########.#.#.###.#.#.#.#######.#.#########.#.#.#.###.#.#.#.#.#.###.#.#.#.#############.#.#.#.#.#########.#.#########.###.#.#.#.###.#
#.#.#.#...#...#...#.#...#.#...#.#.....#...#...#...#.#.#.#.....#.#...#.#...#...#.#.................#.#.#.........#.#.........#.#...#...#.#...#
#.#.#.#.#.#.#.#.###.###.#.#####.#####.#.###.#.#.#.#.#.#.#.###########.###.#####.#############.#####.#.###.###.#.#.###.#####.#.#.###.###.#.#.#
#.#.#.....#.#...#.....#...#...#.....#.#.....#...#...#.#.#...........#.....#.........#.....#...#...#.#...#.....#.#...#...#.#...#.#...#...#.#.#
#.#######.#.#####.###.#####.#######.#.#################.#########.###.#####.#######.#.###.#.#.#.#.#.###.#.#.###.###.###.#.#####.#.###.###.#.#
#.......#.#.....#.....#...#.......#...#...#...#.........#.....#...#.........#.........#...#.#.#.#.#.#...#.....#...#.....#.......#...#.#...#.#
#.#.###.#.#####.#######.#.#.#.#.#.###.#.#.#.###.###.###.#.###.#.###.#######.#.#######.#####.###.#.###.#######.#.#.#############.###.#.#.###.#
#.#...#.#...#.#.........#...#.#.#.#.#.#.#...#...#.#.#...#...#.#.....#.......#.#.....#.#...#.....#.#...#.......#.#.....#...................#.#
#####.#.###.#.###############.#.#.#.#.#.###.#.###.#.#.#######.#########.#####.#.#.#.###.#.###.#.#.#.###.#####.#.#####.#.#######.#########.#.#
#.....#.#...#.........#...#...#.....#.#.#...#...#.#.#.......#.........#...#.#...#.#.....#...#...#.....#.#...#.#...#.........#...#.......#...#
#.#####.#.###.#.###.#.###.#.#.#######.#.#.#####.#.#.#######.###.###.#.#.#.#.#.###.#########.###.#####.#.#.###.#.#.#.#####.#.#####.#####.#.#.#
#.....#.#.#...#.#.#...#...#.#.#.....#.#...........#.#.....#...#.#...#.#.#.#...#...#.....#...#.......#...#...#.#.#.#.....................#.#.#
#.###.#.#.#.###.#.#.###.###.###.###.#.###.###.#####.###.#####.#.#.###.###.#.###.###.###.#.###.#.###.#####.#.#.#.#.###.#.###########.###.#.#.#
#...#.#.#.#.#...#...#...#...#...#.............................#.#.#.#...#.#.#.#.......#.#...#.....#.#.....#.....#.#.#.#.#.......#...#...#...#
###.#.#.#.###.###.###.#.#.###.###.#####.#.#.###.#.###.#.#.#.#####.#.###.#.#.#.#########.#.#.###.#.#.###########.#.#.#.#.#.#.#####.###.#.###.#
#...#.#.#...#.#.....#.#.#.#...#.........#.......#.....#.#.#...#...#...#...#...#...#.....#.......#.#...........#...#...#.#.#...#...#.......#.#
#####.#.###.#.#####.#.#.#.#.#######.#####.###.#########.#.###.#.###.#########.#.###.#.#.#.#####.#.#.#########.#.#.#####.###.#.#.#####.#.#.#.#
#.....#.#...........................#.....#...#.......#.#...#.#...#.........#.#...#.#...#...#.....#.............#.#...#...#.#.#.....#.#.#.#.#
#.#######.###.#.#.#####.#.###########.###.#.#####.###.#.#.###.###.#####.#.#.#.###.#.#.###.#.#######.#.###.#.###.###.#.###.###.#####.#.#.#.#.#
#.#.............#.....#.#.....#.....#.....#.....#...#.#.#...#.....#...#.#.#.#.#...#.......#...#...#.#.#...#.#...#.................#.#...#.#.#
#.#.###########.#.###.#.###.#.#.###.#.###.#.###.#####.#.###.#######.#.###.#.#.#.#.###.###.###.###.#.#.###.#.#.#.#.###.###.#.#####.#.#.#.###.#
#...#.....#.....#.#.#...#...#.#.#.#.#...#.....#.....#.#.#.#.#.......#.....#.#...#...#.......#...#...#.....#.........#.#...#.#.....#.#.#.....#
#.#####.#.#.#####.#.#####.#####.#.#.###.###########.#.#.#.#.#.###.#.###############.#.###.#.###.#########.#.#.#.###.#.#.###.#.#####.###.#####
#S......#...#...........#.........#.................#.....#.......#...........................#...........#.................#.....#.........#
#############################################################################################################################################"""
    maze = [list(line) for line in sample_maze.splitlines()]
    solver = MazeSolver(maze)
    result = solver.find_optimal_tiles()
    print(f"Number of tiles in optimal paths: {result}")

if __name__ == "__main__":
    main()

Number of tiles in optimal paths: 449
